    语音记录包含了由墙壁反射造成的回声。采用自相关技术可将其滤除。在录音文件中，可以听到一个人清晰地说出了“MATLAB”这个单词。相应的音频数据已按照7418Hz的采样率加载。

In [6]:
using MAT
# using Plots
using TyPlot
using TySignalProcessing
using TyMath
using TyBase
using TyStatistics

matfile = matread("mtlb.mat")
mtlb = matfile["mtlb"]


4001×1 Matrix{Float64}:
  0.006535245327923146
 -0.007809729198053787
 -0.055053584205911395
 -0.07122847912711133
 -0.10695453589806567
 -0.11916792188665987
 -0.11572129532886535
 -0.116275588191028
 -0.09216382564039872
 -0.0725532038120437
  ⋮
  0.02322347875636209
  0.025328183037200136
  0.00852113904958042
 -0.006720118533521406
 -0.06966164839492528
 -0.09536450347585694
 -0.10517954035072741
 -0.12684989822010082
 -0.13990038108936506

    为了模拟录音中的回声效果，我们采用了以下模型：将原始信号x(n)与其经过时延Δ和衰减α的副本相加，得到处理后的信号y(n)。具体地，我们设定了0.23秒的时延和0.5的衰减系数，因此模型可以表示为：

$$y(n) = x(n) + 0.5 \times x(n - \Delta)$$

    其中 $\Delta$ 对应于0.23秒的时延在采样率下的样本数。

In [7]:

Fs = 7418
timelag = 0.23;
delta = round(Int, Fs * timelag);
alpha = 0.5;

orig = [mtlb; zeros(delta, 1)];
echo = [zeros(delta, 1); mtlb] * alpha;

mtEcho = orig + echo;
t = (0:length(mtEcho)-1) / Fs;


    执行代码，绘制原始信号、回波信号和结果信号。

In [8]:
figure(1)
subplot(2, 1, 1);
plot(t, [orig echo])
legend("Original", "Echo")

subplot(2, 1, 2)
plot(t, mtEcho)
legend("Total")
xlabel("Time (s)")

PyObject <objects.mw_text.CXlabel object at 0x000002386CAF8788>

    为了获得信号自相关的无偏估计，我们执行了相应的计算。随后，我们选取并展示了自相关函数中滞后大于零的部分，以便于进一步分析和理解信号的时序特性。

In [9]:
Rmm, lags = xcorr(mtEcho; scale="unbiased")

Rmm = Rmm[lags.>0];
lags = lags[lags.>0];

figure(2)
plot(lags / Fs, Rmm)
xlabel("Lag (s)")

PyObject <objects.mw_text.CXlabel object at 0x0000023875551588>

自相关函数在回波到达的特定滞后时间点显示出明显的峰值。为了消除这一回声干扰，我们采用了一个无限冲击响应（IIR）滤波系统对信号进行处理。该系统的输出 $w(n)$ 遵循如下差分方程：

$$w(n) + \alpha w(n-\Delta) = y(n)$$

其中 $\alpha$ 表示衰减系数，$\Delta$ 是回波的时延。通过这种滤波方式，我们能够有效地去除录音中的回声成分。


In [10]:
~, dl = findpeaks(Rmm, lags; MinPeakHeight=0.22);
mtNew, = filter1(1, [1;zeros(dl[1] - 1, 1); alpha], mtEcho);

figure(3)
subplot(2, 1, 1)
plot(t, orig)
legend("Original")

subplot(2, 1, 2)
plot(t, mtNew)
legend("Filtered")
xlabel("Time (s)")

PyObject <objects.mw_text.CXlabel object at 0x0000023876D5B2C8>